<a href="https://colab.research.google.com/github/SunmoKoo/research-prac/blob/main/vgg19_TransferLearning(2_8%2C_data_ext).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, models, layers, optimizers, metrics
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/SunmoKoo/research-prac.git

fatal: destination path 'research-prac' already exists and is not an empty directory.


In [ ]:
data_generator = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True, zoom_range=1.2, fill_mode='nearest', validation_split=0.8)
train_generator = data_generator.flow_from_directory('research-prac/data_prac/data', target_size=(240,240), batch_size=8, class_mode='binary')
val_generator = data_generator.flow_from_directory('research-prac/data_prac/data', target_size=(240,240), batch_size=8, class_mode='binary')

Found 553 images belonging to 2 classes.
Found 553 images belonging to 2 classes.


In [ ]:
transfer_model = VGG19(weights='imagenet', include_top=False, input_shape=(240,240,3))
transfer_model.trainable = False

In [ ]:
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(256))
finetune_model.add(Activation('relu'))
finetune_model.add(Dropout(0.5))
finetune_model.add(Dense(1))
finetune_model.add(Activation('sigmoid'))
finetune_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
 activation_1 (Activation)   (None, 1)                 0

In [ ]:
finetune_model.compile(loss='binary_crossentropy', optimizer = optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=16)

In [ ]:
history = finetune_model.fit(train_generator, epochs=200, validation_data=val_generator, callbacks=[early_stopping_callback])

Epoch 1/200
70/70 [==============================] - 853s 12s/step - loss: 0.4467 - accuracy: 0.8662 - val_loss: 0.4133 - val_accuracy: 0.8807
Epoch 2/200
70/70 [==============================] - 929s 13s/step - loss: 0.3935 - accuracy: 0.8770 - val_loss: 0.3497 - val_accuracy: 0.8807
Epoch 3/200
70/70 [==============================] - 991s 14s/step - loss: 0.3533 - accuracy: 0.8788 - val_loss: 0.3641 - val_accuracy: 0.8807
Epoch 4/200
70/70 [==============================] - 865s 12s/step - loss: 0.3648 - accuracy: 0.8807 - val_loss: 0.3346 - val_accuracy: 0.8807
Epoch 5/200
70/70 [==============================] - 867s 12s/step - loss: 0.3685 - accuracy: 0.8807 - val_loss: 0.3420 - val_accuracy: 0.8807
Epoch 6/200
70/70 [==============================] - 868s 12s/step - loss: 0.3371 - accuracy: 0.8807 - val_loss: 0.3405 - val_accuracy: 0.8807
Epoch 7/200
70/70 [==============================] - 846s 12s/step - loss: 0.3634 - accuracy: 0.8807 - val_loss: 0.3325 - val_accuracy: 0.8807

In [ ]:
y_vacc = history.history['val_accuracy']
y_acc = history.history['accuracy']

x_len = np.arange(len(y_acc))
plt.plot(x_len, y_vacc, marker='.', c='red', label='val_accuracy')
plt.plot(x_len, y_acc, marker='.', c='blue', label = 'accuracy')

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label='val_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label = 'loss')

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show